In [8]:
import importlib
import xarray as xr
import numpy as np

In [2]:
pathout="/project/cas/islas/python_savs/snowpaper/DATA_SORT/3cities/"

In [3]:
cityname=['Saskatoon','Toronto','Siderovsk']
citylon=[253.330, 280.617, 82.3139]
citylat=[52.1579, 43.6532, 66.5973]

In [5]:
data = xr.open_dataset("/project/cas/islas/savs/python/ISD/T2M_ISD_1979_2014.nc")
t2m = data['t2m']
lon = data['lon']
lat = data['lat']

In [6]:
# !!!!!! convert to celcius
t2m = (t2m - 32)/1.8

In [9]:
# only use the stations that have data on more than 80% of the days
numnans = [np.count_nonzero(np.isnan(t2m[i,:])) for i in range(0,t2m['station'].size,1)]
mask = (np.array(numnans) < (0.8*t2m.time.size))
t2m = t2m[mask,:] ; lon = lon[mask] ; lat = lat[mask]

In [11]:
lon[(lon < 0)] = lon[(lon < 0)] + 360

In [25]:
isdtrefht = xr.DataArray(np.zeros([isd.time.size,3]), coords = [isd.time,cityname], dims=['time','city'], name='isd')
for icity in np.arange(0,len(cityname),1):
    distance = np.abs(lon-citylon[icity]) + np.abs(lat-citylat[icity])
    index = np.argmin(np.array(distance))
    isdtrefht[:,icity] = t2m[index, :]
    print(lon[index],lat[index])

<xarray.DataArray 'lon' ()>
array(253.3)
Coordinates:
    station  <U11 '71866099999' <xarray.DataArray 'lat' ()>
array(52.167)
Coordinates:
    station  <U11 '71866099999'
<xarray.DataArray 'lon' ()>
array(280.369)
Coordinates:
    station  <U11 '71624099999' <xarray.DataArray 'lat' ()>
array(43.677)
Coordinates:
    station  <U11 '71624099999'
<xarray.DataArray 'lon' ()>
array(84.328)
Coordinates:
    station  <U11 '23463099999' <xarray.DataArray 'lat' ()>
array(65.983)
Coordinates:
    station  <U11 '23463099999'


In [26]:
isdtrefht.to_netcdf(path=pathout+"ISD_TREFHT.nc")